For  all the non-proper noun keys, we need to create embeddings.


In [1]:
#| default_exp create_embeddings


We create a new table with the keys that require embeddings.

In [18]:
from hn_jobs_chat import keys
from hn_jobs_chat.var import postsTableName
from hn_jobs_chat.db import connectToDB

conn, cursor = connectToDB()

# Create a new table with the keys that are embeddable

embeddable_keys = []
for key in keys.described_keys:
    if key['embed'] and key['type'] == 'string':
        embeddable_keys.append(key)

embeddable_keys_str = ' vector(1536), '.join([key['key'] for key in embeddable_keys])

embeddable_keys_str = embeddable_keys_str + ' vector(1536)'

query = """CREATE TABLE IF NOT EXISTS """ + postsTableName + """_embeddings (
        id SERIAL PRIMARY KEY, 
        """ + embeddable_keys_str + """
        );"""

cursor.execute(query)
conn.commit()
conn.close()


Create the embeddings

In [21]:
from openai import OpenAI

from hn_jobs_chat.var import postsTableName
from hn_jobs_chat.db import connectToDB

client = OpenAI()

conn, cursor = connectToDB()

query = """SELECT json_agg("""+ postsTableName +""")  FROM """ + postsTableName
cursor.execute(query)

result = cursor.fetchall()

data = result[0][0]

embeddingTableName = postsTableName + "_embeddings"

for datum in data:

    item_id = datum['id']

    query = "INSERT INTO " + embeddingTableName + " (id) VALUES (%s)"
    cursor.execute(query, (item_id,))
    conn.commit()

    for key in embeddable_keys:
        text = datum[key['key']]

        if text is not None:
            response = client.embeddings.create(
                input=text,
                model="text-embedding-3-small"
            )

            embedding = (response.data[0].embedding)

            query = """UPDATE """ + embeddingTableName + """ SET """ + key['key'] + """ = %s::vector(1536) WHERE id = %s"""

            cursor.execute(query, (embedding, item_id))
            conn.commit()

    print('processed: ' + str(item_id))
    
conn.close()


processed: 30
processed: 114
processed: 46
processed: 148
processed: 31
processed: 149
processed: 63
processed: 47
processed: 103
processed: 115
processed: 41
processed: 124
processed: 13
processed: 64
processed: 65
processed: 74
processed: 150
processed: 48
processed: 7
processed: 66
processed: 104
processed: 116
processed: 75
processed: 49
processed: 96
processed: 140
processed: 14
processed: 1
processed: 67
processed: 15
processed: 68
processed: 105
processed: 151
processed: 50
processed: 132
processed: 97
processed: 69
processed: 16
processed: 90
processed: 141
processed: 32
processed: 81
processed: 2
processed: 142
processed: 17
processed: 58
processed: 82
processed: 98
processed: 23
processed: 83
processed: 76
processed: 91
processed: 125
processed: 126
processed: 18
processed: 19
processed: 127
processed: 24
processed: 3
processed: 133
processed: 33
processed: 42
processed: 92
processed: 84
processed: 4
processed: 25
processed: 99
processed: 20
processed: 34
processed: 5
process

In [11]:
embeddingStr

'0.045360811054706573,-0.014795861206948757,-0.05929628387093544,0.0022109164856374264,-0.003351636463776231,0.04521976411342621,-0.0024894848465919495,0.04248344525694847,-0.002225021133199334,-0.027546538040041924,0.0009626477840356529,-0.020324915647506714,0.017433445900678635,0.012743622995913029,-0.005578420590609312,0.04493767023086548,-0.07266756892204285,-0.014457347802817822,0.046235304325819016,0.00512706907466054,0.011664611287415028,0.025346199050545692,-0.033202532678842545,0.02279324270784855,-0.0192529559135437,-0.01609349623322487,-0.01953505165874958,0.008561571128666401,0.03342821076512337,-0.010366976261138916,0.0385059118270874,-0.045078717172145844,0.019450422376394272,-0.03373851254582405,-0.016996199265122414,-0.014189357869327068,-0.04019847884774208,0.04364003241062164,-0.021481502801179886,0.01629096269607544,0.050410304218530655,-0.04730726405978203,0.030861148610711098,0.016135811805725098,0.027179814875125885,0.030635472387075424,-0.0442042239010334,-0.0367